## 4. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Preliminaries

In [ ]:
import os
from pathlib import Path

# The path at which media is stored
REPORTS_DIR = Path(os.getcwd()) / "reports"
os.makedirs(REPORTS_DIR, exist_ok=True)

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [ ]:
from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")

set_context("IrisClassifier", "0.0.1")
set_store(f"local://{store_path}")

### Validate Values and get an updated `ValidatedSpec` with `Result`s

Now that we have our `Spec` ready and we have enough evidence, we create a `SpecValidator` with our spec, and add all the `Value`s we have. With that we can validate our spec and generate an output `ValidatedSpec`, with the validation results.

In [ ]:
from mlte.tests.test_suite import TestSuite
from mlte.validation.test_suite_validator import TestSuiteValidator
from mlte.evidence.artifact import Evidence

# Load the test suite
test_suite = TestSuite.load()

# Add all evidence to the validator.
test_suite_validators = TestSuiteValidator(test_suite)
test_suite_validators.add_evidence_list(Evidence.load_all())

In [ ]:
# Validate requirements and get validated details.
test_results = test_suite_validators.validate()

# ValidatedSpec also supports persistence
test_results.save(force=True)

#### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [ ]:
from mlte.report.artifact import (
    Report,
    CommentDescriptor,
    QuantitiveAnalysisDescriptor,
)
from mlte.negotiation.artifact import NegotiationCard

report = Report(
    test_results_id=test_results.identifier,
    comments=[
        CommentDescriptor(
            content="This model should not be used for nefarious purposes."
        )
    ],
    quantitative_analysis=QuantitiveAnalysisDescriptor(
        content="Insert graph here."
    ),
)

negotiation_card = NegotiationCard.load()
report = report.populate_from(negotiation_card)

report.save(force=True, parents=True)